In [1]:
# !pip install wandb

In [1]:
import wandb
import torch
import torch.nn as nn
from torch.nn import functional as F
from torcheval.metrics.text import Perplexity
from tqdm import tqdm
from attention import GPTLanguageModel
from mamba import MambaLanguageModel
from xlstm import XLSTMLanguageModel
torch.manual_seed(1337)
import gc

In [2]:
# !wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

In [3]:
wandb.login()

wandb: Currently logged in as: shetty-sau (shetty-sau-northeastern-university). Use `wandb login --relogin` to force relogin


True

In [4]:
batch_size = 125 # how many independent sequences will we process in parallel?
block_size = 256 # what is the maximum context length for predictions?
max_iters = 500000
eval_interval = 500
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 384
n_head = 6
n_layer = 6
dropout = 0.2
torch.set_default_device(device)
device

'cuda'

In [5]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)

# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [6]:
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

In [7]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        # metric = Perplexity()
        # metric.to(device)
        # metric.reset()
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            # BT, C = logits.shape
            # logits = logits.view(batch_size, BT//batch_size, C)
            # Y = Y.view(batch_size, BT//batch_size)
            # metric.update(logits, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
        # out[str(split)+"_perplexity"] = metric.compute()
        # del metric
    model.train()
    return out
    
# @torch.no_grad()
# def estimate_loss():
#     out = {}
#     model.eval()
#     for split in ['train', 'val']:
#         losses = torch.zeros(eval_iters)
#         for k in range(eval_iters):
#             X, Y = get_batch(split)
#             logits, loss = model(X, Y)
#             losses[k] = loss.item()
#         out[split] = losses.mean()

#     for split in ['perplexity_train', 'perplexity_val']:
#         metric = Perplexity()
#         metric.to(device)
#         metric.reset()
#         for k in range(eval_iters):
#             X, Y = get_batch(split)
#             logits, loss = model(X, Y)
#             BT, C = logits.shape
#             logits = logits.view(batch_size, BT//batch_size, C)
#             Y = Y.view(batch_size, BT//batch_size)
#             metric.update(logits, Y)
#         out[split] = metric.compute()
#         del metric
#     model.train()
#     return out

In [8]:
model = GPTLanguageModel(vocab_size)
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

KeyboardInterrupt: 

In [9]:
# Parameters for early stopping
wandb.watch(model, optimizer, log="all", log_freq=100)

patience = 5
min_delta = 0.001
best_val_loss = float('inf')
patience_counter = 0

for iter in tqdm(range(max_iters)):
    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f} train_perplexity {losses['train_perplexity']:.4f}, val_perplexity  {losses['val_perplexity']:.4f}")
        wandb.log({"train_loss": losses['train'], "val_loss": losses['val'], "train_perplexity":losses['train_perplexity'], "val_perplexity":losses['val_perplexity']})

        # Early stopping check
        if losses['val'] < best_val_loss - min_delta:
            best_val_loss = losses['val']
            patience_counter = 0
        else:
            patience_counter += 1

        if patience_counter >= patience:
            print(f"Early stopping at iteration {iter} with best val loss {best_val_loss:.4f}")
            break

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

  0%|          | 0/500000 [00:00<?, ?it/s]

step 0: train loss 4.3219, val loss 4.3331 train_perplexity 75.3341, val_perplexity  76.1789


  0%|          | 500/500000 [09:07<127:21:10,  1.09it/s] 

step 500: train loss 1.5949, val loss 1.7762 train_perplexity 4.9279, val_perplexity  5.9072


  0%|          | 1000/500000 [18:17<126:32:39,  1.10it/s]

...

step 3000: train loss 0.9117, val loss 1.5317 train_perplexity 2.4887, val_perplexity  4.6260


  1%|          | 3500/500000 [1:03:56<124:56:34,  1.10it/s]

step 3500: train loss 0.8318, val loss 1.5840 train_perplexity 2.2974, val_perplexity  4.8742


  1%|          | 4000/500000 [1:15:12<155:26:31,  1.13s/it] 

step 4000: train loss 0.7435, val loss 1.6289 train_perplexity 2.1032, val_perplexity  5.0982
Early stopping at iteration 4000 with best val loss 1.4867


In [10]:
# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))


Black, pioous, use! my gracious right!
That fearful worms' tears will under dryck.

ApoLIXENES:
Pray you, then, let's to see.

ESCALUS:
Where is threat joy? why, sir?

Clown:
Army is't like a sect father's enemy.

ANGELO:
You smell of spoken?

ESCALUS:
No. Well, sir, I like it.

LUCIO:
Good villana.

POMPEY:
Pray, daught you think; and, I can, if you, pray 
by unrespect, or any of the packer is life, I thou shall
goe of your corner, you'll go jest me to unto the noble
of my comfort, and lear me 


# MAMBA

In [9]:
# clear GPU memory
del model
del m
gc.collect()
torch.cuda.empty_cache() 

NameError: name 'model' is not defined

In [8]:
batch_size = 125 # how many independent sequences will we process in parallel?
block_size = 256 # what is the maximum context length for predictions?
max_iters = 500000
eval_interval = 500
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 384
n_head = 6
n_layer = 6
dropout = 0.2

model = MambaLanguageModel(vocab_size, batch_size = batch_size, block_size = block_size,
                max_iters = max_iters, eval_interval = eval_interval, learning_rate = learning_rate,
                n_embd = n_embd, n_head = n_head,
                n_layer = n_layer, dropout = dropout, device = device)

m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

5.933633 M parameters


In [9]:
wandb.watch(model, optimizer, log="all", log_freq=100)

# Parameters for early stopping
patience = 5
min_delta = 0.001
best_val_loss = float('inf')
patience_counter = 0

for iter in tqdm(range(max_iters)):
    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f} train_perplexity {losses['train_perplexity']:.4f}, val_perplexity  {losses['val_perplexity']:.4f}")
        wandb.log({"train_loss": losses['train'], "val_loss": losses['val'], "train_perplexity":losses['train_perplexity'], "val_perplexity":losses['val_perplexity']})

        # Early stopping check
        if losses['val'] < best_val_loss - min_delta:
            best_val_loss = losses['val']
            patience_counter = 0
        else:
            patience_counter += 1

        if patience_counter >= patience:
            print(f"Early stopping at iteration {iter} with best val loss {best_val_loss:.4f}")
            break

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

  0%|                                                                                                            | 0/500000 [00:04<?, ?it/s]


RuntimeError: CUDA error: the launch timed out and was terminated
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))

# xLSTM

In [ ]:
# clear GPU memory
del model
delet m
gc.collect()
torch.cuda.empty_cache() 

In [8]:
batch_size = 125 # how many independent sequences will we process in parallel?
block_size = 256 # what is the maximum context length for predictions?
max_iters = 500000
eval_interval = 500
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
layer = ['m', 'm', 's']
dropout = 0.2
torch.set_default_device(device)

x = torch.zeros(batch_size, block_size, n_embd)

model = XLSTMLanguageModel(vocab_size, x, batch_size = batch_size, block_size = block_size,
                max_iters = max_iters, eval_interval = eval_interval, learning_rate = learning_rate,
                device = device, eval_iters = eval_interval, dropout = dropout, layers=layer)

m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

63.902017 M parameters


In [9]:
wandb.watch(model, optimizer, log="all", log_freq=100)

# Parameters for early stopping
patience = 5
min_delta = 0.001
best_val_loss = float('inf')
patience_counter = 0

for iter in tqdm(range(max_iters)):
    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f} train_perplexity {losses['train_perplexity']:.4f}, val_perplexity  {losses['val_perplexity']:.4f}")
        wandb.log({"train_loss": losses['train'], "val_loss": losses['val'], "train_perplexity":losses['train_perplexity'], "val_perplexity":losses['val_perplexity']})


        # Early stopping check
        if losses['val'] < best_val_loss - min_delta:
            best_val_loss = losses['val']
            patience_counter = 0
        else:
            patience_counter += 1

        if patience_counter >= patience:
            print(f"Early stopping at iteration {iter} with best val loss {best_val_loss:.4f}")
            break

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

  0%|          | 0/500000 [00:00<?, ?it/s]

step 0: train loss 4.3027, val loss 4.3005 train_perplexity 73.8967, val_perplexity  73.7376


  0%|          | 500/500000 [22:35<368:48:33,  2.66s/it] 

step 500: train loss 2.5712, val loss 2.6183 train_perplexity 13.0820, val_perplexity  13.7124


  0%|          | 1000/500000 [45:12<369:18:06,  2.66s/it]

...

step 5500: train loss 2.4566, val loss 2.4854 train_perplexity 11.6654, val_perplexity  12.0054


  1%|          | 6000/500000 [4:31:24<365:59:29,  2.67s/it] 

step 6000: train loss 2.4553, val loss 2.4839 train_perplexity 11.6503, val_perplexity  11.9885


  1%|▏         | 6500/500000 [4:57:09<376:01:18,  2.74s/it] 

step 6500: train loss 2.4554, val loss 2.4849 train_perplexity 11.6506, val_perplexity  11.9999
Early stopping at iteration 6500 with best val loss 2.4839


In [12]:
# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))

TypeError: unhashable type: 'list'

# Evaluation

!pip install datasets

In [ ]:
import torch
import datasets
from torcheval.metrics.text import Perplexity
block_size = 256
device = 'cuda' if torch.cuda.is_available() else 'cpu'
batch_size = 32
next_token_len = 100
metric = datasets.load_metric('bleu', trust_remote_code=True)
perplexity_metric = Perplexity()
perplexity_metric.to(device)

In [ ]:
def get_bleu_batch(max_tokens = 10):
    data = val_data
    ix = torch.randint(len(data) - block_size - max_tokens, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+block_size:i+block_size+max_tokens] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_bleu_batch(next_token_len)

In [ ]:
attention_model = torch.load('/content/attention_best_model.pt')
mamba_model = torch.load('/content/mamba_best_model.pt')
xlstm_model = torch.load('/content/xlstm_best_model.pt')
attention_model = attention_model.to(device)
mamba_model = mamba_model.to(device)
xlstm_model = xlstm_model.to(device)


In [ ]:

print("Attention model - ",sum(p.numel() for p in attention_model.parameters())/1e6, 'M parameters')
print("MAMBA model - ",sum(p.numel() for p in mamba_model.parameters())/1e6, 'M parameters')
'''
Attention model -  10.788929 M parameters
MAMBA model -  5.936705 M parameters
'''

In [ ]:
for name, m in [("Attention model", attention_model), ("MAMBA model", mamba_model)]:
    m.eval()
    r = m.generate(x, max_new_tokens=next_token_len)
    predictions = [list(decode(i)) for i in r[:, block_size:].cpu().numpy()]
    references = [[list(decode(i))] for i in y.cpu().numpy()]
    results = metric.compute(predictions=predictions, references=references)

    ## Calculate perplexity

    perplexity_metric.reset()
    for k in range(eval_iters):
        X, Y = get_batch('val')
        logits, loss = m(X, Y)
        BT, C = logits.shape
        logits = logits.view(batch_size, BT//batch_size, C)
        Y = Y.view(batch_size, BT//batch_size)
        perplexity_metric.update(logits, Y)
    print(name)
    print(results)
    print("Perplexity - ",perplexity_metric.compute())
    print('*'*20)
    print("INPUT")
    print(''.join(list(decode(x.cpu().numpy()[0]))))  
    print("REFERENCE")
    print(''.join(references[0][0]))
    print("PREDICTION")
    print(''.join(predictions[0]))
    print('*'*20)

Attention model
{'bleu': 0.1620102294221506, 'precisions': [0.69875, 0.28724747474747475, 0.09598214285714286, 0.03576030927835051], 'brevity_penalty': 1.0, 'length_ratio': 1.0, 'translation_length': 3200, 'reference_length': 3200}
Perplexity -  tensor(4.8975, device='cuda:0', dtype=torch.float64)
********************
INPUT
nd therefore here I mean to take my leave.

BAPTISTA:
Is't possible you will away to-night?

PETRUCHIO:
I must away to-day, before night come:
Make it no wonder; if you knew my business,
You would entreat me rather go than stay.
And, honest company, I than
REFERENCE
k you all,
That have beheld me give away myself
To this most patient, sweet and virtuous wife:
Dine 
PREDICTION
k your grace
Strave, yet for you, fill the
That certable have your brother grief truth bavison; yea.
********************
MAMBA model
{'bleu': 0.17078674925312282, 'precisions': [0.7165625, 0.3020833333333333, 0.10427295918367346, 0.03769329896907216], 'brevity_penalty': 1.0, 'length_ratio': 1.0, 'translation_length': 3200, 'reference_length': 3200}
Perplexity -  tensor(4.7281, device='cuda:0', dtype=torch.float64)
********************
INPUT
nd therefore here I mean to take my leave.

BAPTISTA:
Is't possible you will away to-night?

PETRUCHIO:
I must away to-day, before night come:
Make it no wonder; if you knew my business,
You would entreat me rather go than stay.
And, honest company, I than
REFERENCE
k you all,
That have beheld me give away myself
To this most patient, sweet and virtuous wife:
Dine 
PREDICTION
k thee, man:
Alas, when I had a voice of the mean
As is the like rebukes, such a other:
Alack, that 
********************